In [1]:
import numpy as np
import cv2
import yaml
from hsi_dataset_api import HsiDataCropper, HsiDataset
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [2]:
dataset = HsiDataset(r'.\DataSet\RC0', cropped_dataset=True)

Let's work only with specters that contain 'potato' class

In [3]:
specters_with_potato = []
for data_point in dataset.data_iterator(opened=False):
    specter = data_point.hsi
    mask = data_point.mask
    meta = data_point.meta
    with open(meta, 'r') as f:
        meta = yaml.full_load(f)
    data_point.meta = meta
    if 'potato' in meta['classes']:
        specters_with_potato.append(data_point)

In [4]:
len(specters_with_potato)

40

This array contains shuffled indices from 0 to length of specters_with_potato

In [5]:
choose_array = np.random.permutation(len(specters_with_potato))

In [6]:
xtrain, xtest, ytrain, ytest = [], [], [], []

In the following cells we create train and test data by taking 20 specters for each sampling.

From each specter we take 1% of vectors randomly.

In [7]:
for idx in choose_array[:20]:
    data_point = specters_with_potato[idx]
    specter = np.load(data_point.hsi)
    mask = cv2.imread(data_point.mask)[..., 0]
    meta = data_point.meta

    for i in range(int(512 * 512 * 0.01)):
        y = np.random.randint(0, 512)
        x = np.random.randint(0, 512)
        vec = specter[::, y, x]

        xtrain.append(vec)
        ytrain.append(mask[y, x])

In [8]:
for idx in choose_array[20:40]:
    data_point = specters_with_potato[idx]
    specter = np.load(data_point.hsi)
    mask = cv2.imread(data_point.mask)[..., 0]
    meta = data_point.meta

    for i in range(int(512 * 512 * 0.01)):
        y = np.random.randint(0, 512)
        x = np.random.randint(0, 512)
        vec = specter[::, y, x]

        xtest.append(vec)
        ytest.append(mask[y, x])

Let's train simple multilayer perceptron

In [9]:
clf = MLPClassifier(random_state=0, max_iter=10_000)
clf.fit(xtrain, ytrain)

MLPClassifier(max_iter=10000, random_state=0)

In [10]:
preds = clf.predict(xtest)

In [11]:
print(classification_report(ytest, preds))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96     23745
           1       0.85      0.83      0.84     24455
           2       0.18      0.17      0.18      4220

    accuracy                           0.84     52420
   macro avg       0.66      0.66      0.66     52420
weighted avg       0.84      0.84      0.84     52420

